Enclosed dataset contains data about press headlines – their content and type. Aim of this task is to create binary classification model for headline type (sarcastic / not sarcastic) based on headline content.

Goal: Based on attached data build a model that will classify headline types. Prepare a report where you describe your way of approaching the problem and the steps you took to solve it. Don’t forget to assess the quality of the model you have prepared.

Don’t focus on getting the highest score possible - the more important thing is to show your line of thought and your approach to the problem - especially NLP techniques you used.

If you have any doubts or something seems inconclusive based on the task description, write it down in the report in the way you would ask the customer to clarify.
